## Lin-based Models

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from datasets import Dataset
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer
from transformers import TrainingArguments, Trainer
from sklearn import metrics
from tensorflow import keras
from keras.layers import Dense, Dropout,TextVectorization, Flatten, Concatenate
from keras import Model
import keras
import tensorflow as tf
import evaluate

/opt/anaconda3/envs/discoursetbi/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Data Processing

In [2]:
lin_data = pd.read_csv('lin.csv')

In [3]:
lin_data = lin_data.drop("Unnamed: 0", axis=1)
lin_data['labels'] = lin_data['labels'].astype('category')
lin_data['labels'] = lin_data['labels'].cat.codes

In [5]:
X1= lin_data[['Explicit', 'ComparisonConcession', 'ContingencyCauseReason', 'ContingencyCondition', 'Implicit', 
              'ExpansionRestatement', 'ComparisonContrast', 'Null', 'EntRel', 'ExpansionConjunction', 'ContingencyCauseResult']].values
X2 = lin_data['text']
Y = lin_data['labels'].values    

X1_train, X1_test, X2_train, X2_test, y_train, y_test = train_test_split(X1, X2, Y, train_size=0.75, random_state= 42)

In [6]:
y_test = to_categorical(y_test)
y_train = to_categorical(y_train)

In [7]:
train2_samples = [x for x in X2_train]
train_labels = [y for y in y_train]

test2_samples = [x for x in X2_test]
test_labels = [y for y in y_test]

In [8]:
# Vectorize texts
vectorizer = TextVectorization(max_tokens=1000, output_mode= 'multi_hot', pad_to_max_tokens=True)
vectorizer.adapt(train2_samples)
voc = vectorizer.get_vocabulary()

In [9]:
x_train2 = vectorizer(np.array([[s] for s in train2_samples]))
x_test2  = vectorizer(np.array([[s] for s in test2_samples]))

x_train1 = np.asarray(X1_train)
x_test1 = np.asarray(X1_test)
y_train = np.array(train_labels)
y_test = np.array(test_labels)

### Basic RNN Model

In [10]:
pdtb_input = keras.Input((11,))
text_input = keras.Input((1000,))
x1 = Flatten()(pdtb_input)
x2 = Flatten()(text_input)
x = Concatenate()([x1, x2])
x = Dense(16, activation='relu')(x)
x = Dropout(0.2)(x)
output = Dense(2, activation='sigmoid')(x)
model = Model(inputs = [pdtb_input, text_input], outputs= output)
model.summary()

callback = keras.callbacks.EarlyStopping()
opt = keras.optimizers.AdamW(learning_rate=0.05)
model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["binary_accuracy", "f1_score"])
model.fit([x_train1, x_train2], y_train, epochs=10, batch_size=32, validation_data=([x_test1, x_test2], y_test), callbacks=[callback])

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 11)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, 1000)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 11)        │          0 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_1 (Flatten) │ (None, 1000)      │          0 │ input_layer_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 1011)      │          0 │ flatten[0][0],    │
│ (Concatenate)       │                   │            │ flatten_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 16)        │     16,192 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 16)        │          0 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 2)         │         34 │ dropout[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 16,226 (63.38 KB)

 Trainable params: 16,226 (63.38 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10


/opt/anaconda3/envs/discoursetbi/lib/python3.9/site-packages/keras/src/models/functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['keras_tensor', 'keras_tensor_1']. Received: the structure of inputs=('*', '*')
  warnings.warn(


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step - binary_accuracy: 0.5052 - f1_score: 0.4583 - loss: 0.9447 - val_binary_accuracy: 0.5000 - val_f1_score: 0.2800 - val_loss: 1.1019
Epoch 2/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - binary_accuracy: 0.7155 - f1_score: 0.8055 - loss: 0.5887 - val_binary_accuracy: 0.4722 - val_f1_score: 0.3750 - val_loss: 0.8302
Epoch 3/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - binary_accuracy: 0.9190 - f1_score: 0.9121 - loss: 0.4210 - val_binary_accuracy: 0.4722 - val_f1_score: 0.6000 - val_loss: 0.8049
Epoch 4/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - binary_accuracy: 0.9452 - f1_score: 0.9627 - loss: 0.2716 - val_binary_accuracy: 0.5833 - val_f1_score: 0.5556 - val_loss: 0.8109


### Transformer Model

In [13]:
dataset = Dataset.from_pandas(lin_data)

In [14]:
model = AutoModelForSequenceClassification.from_pretrained("google-bert/bert-base-cased", num_labels=2, torch_dtype="auto")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-cased")


def tokenize_function(examples):

    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.train_test_split(test_size=0.25)

Map: 100%|██████████| 69/69 [00:00<00:00, 2615.12 examples/s]


In [16]:
metric = evaluate.load("accuracy")

In [17]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [18]:
training_args = TrainingArguments(
    output_dir="test_trainer", 
    eval_strategy="epoch",
)

In [19]:
print(tokenized_datasets)

DatasetDict({
    train: Dataset({
        features: ['text', 'labels', 'Explicit', 'ComparisonConcession', 'ContingencyCauseReason', 'ContingencyCondition', 'Implicit', 'ExpansionRestatement', 'ComparisonContrast', 'Null', 'EntRel', 'ExpansionConjunction', 'ContingencyCauseResult', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 51
    })
    test: Dataset({
        features: ['text', 'labels', 'Explicit', 'ComparisonConcession', 'ContingencyCauseReason', 'ContingencyCondition', 'Implicit', 'ExpansionRestatement', 'ComparisonContrast', 'Null', 'EntRel', 'ExpansionConjunction', 'ContingencyCauseResult', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 18
    })
})


In [20]:
bert_trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset= tokenized_datasets['train'],
    eval_dataset= tokenized_datasets['test'],
    compute_metrics=compute_metrics,
)

In [21]:
bert_trainer.train()

 33%|███▎      | 7/21 [00:12<00:18,  1.32s/it]

{'eval_loss': 0.6827346086502075, 'eval_accuracy': 0.5555555555555556, 'eval_runtime': 1.41, 'eval_samples_per_second': 12.766, 'eval_steps_per_second': 2.128, 'epoch': 1.0}


                                               
 67%|██████▋   | 14/21 [00:24<00:09,  1.41s/it]

{'eval_loss': 0.7075426578521729, 'eval_accuracy': 0.5, 'eval_runtime': 1.4039, 'eval_samples_per_second': 12.821, 'eval_steps_per_second': 2.137, 'epoch': 2.0}


                                               
100%|██████████| 21/21 [00:37<00:00,  1.80s/it]

{'eval_loss': 0.7055848240852356, 'eval_accuracy': 0.5, 'eval_runtime': 1.4074, 'eval_samples_per_second': 12.79, 'eval_steps_per_second': 2.132, 'epoch': 3.0}
{'train_runtime': 37.817, 'train_samples_per_second': 4.046, 'train_steps_per_second': 0.555, 'train_loss': 0.6856163569859096, 'epoch': 3.0}


TrainOutput(global_step=21, training_loss=0.6856163569859096, metrics={'train_runtime': 37.817, 'train_samples_per_second': 4.046, 'train_steps_per_second': 0.555, 'total_flos': 40255991470080.0, 'train_loss': 0.6856163569859096, 'epoch': 3.0})

In [22]:
bert_trainer.evaluate(tokenized_datasets['train'])

100%|██████████| 7/7 [00:02<00:00,  2.65it/s]


{'eval_loss': 0.5647692084312439,
 'eval_accuracy': 0.8823529411764706,
 'eval_runtime': 3.2046,
 'eval_samples_per_second': 15.915,
 'eval_steps_per_second': 2.184,
 'epoch': 3.0}